In [1]:
import pandas as pd

from ch2.data import *
from ch2.squeal import *
from ch2.lib.date import *

s = session('-v 4')

SCORE = 'score'
TEXT = 'text'
ACTIVE_TIME_H = '%s / h' % ACTIVE_TIME

start, finish = to_time('2018-03-01'), to_time('2019-02-28')

t = s.query(Topic).filter(Topic.name == 'Broken Femur LHS').one()
f = [field for field in t.fields if field.statistic_name.name == 'Notes'][0]

sj = s.query(StatisticJournalText).\
    filter(StatisticJournalText.statistic_name == f.statistic_name,
           StatisticJournalText.time >= start,
           StatisticJournalText.time < finish).all()
text = [j.value for j in sj]
time = [j.time for j in sj]
text_data = pd.DataFrame({TEXT: text, TIME: time}, index=time)
wordlist(text_data, TEXT)

health_data = std_health_stats(s, start=start, finish=finish)
health_data[ACTIVE_TIME_H] = health_data[ACTIVE_TIME] / 3600

    INFO: Using database at /home/andrew/.ch2/database.sqlo
/home/andrew/project/ch2/choochoo/env/lib/python3.7/site-packages/pandas/core/indexes/base.py:2819: FutureWarning: Converting timezone-aware DatetimeArray to timezone-naive ndarray with 'datetime64[ns]' dtype. In the future, this will return an ndarray with 'object' dtype where each element is a 'pandas.Timestamp' with the correct 'tz'.
	To accept the future behavior, pass 'dtype=object'.
	To keep the old behavior, pass 'dtype="datetime64[ns]"'.
  target = np.asarray(target)


In [2]:
scorer = WeightedScorer({'little': 0.25, 'less': 0.25, 'some': 0.5, 'slight':0.5, 'mild': 0.5, 
                         'normal': 1, 'burning': 1, 
                         'real': 2, 'sharp': 2, 'strong': 3, 'stabbing': 3}, 
                        {'stiffness': 0.5, 'ache': 1, 'discomfort': 1, 'twinges': 2, 'pain': 3},
                       gamma=0.5)
scorer.score_all(text_data, TEXT, SCORE)
text_data.nlargest(10, SCORE)

,text,Time,score
2019-01-26 03:00:00+00:00,"[some, stabbing, pain, scab, later, afternoon,...",2019-01-26 03:00:00+00:00,3.015113
2018-10-14 03:00:00+00:00,"[discomfort, on, waking, intermittent, pain, a...",2018-10-14 03:00:00+00:00,1.889822
2018-11-02 03:00:00+00:00,"[ache, in, morning, but, did, cleaning, of, ho...",2018-11-02 03:00:00+00:00,1.856953
2018-10-10 03:00:00+00:00,"[little, pain, most, of, day, while, washing, ...",2018-10-10 03:00:00+00:00,1.504458
2018-10-01 03:00:00+00:00,"[did, n't, make, notes, til, tue, so, unsure, ...",2018-10-01 03:00:00+00:00,1.500000
2018-09-18 03:00:00+00:00,"[bum, ache, spreading, to, side, but, scar, pa...",2018-09-18 03:00:00+00:00,1.333333
2018-09-29 03:00:00+00:00,"[good, first, half, of, day, bum, pain, seems,...",2018-09-29 03:00:00+00:00,1.309307
2018-12-02 03:00:00+00:00,"[walked, to, big, jon, lampost, feeling, more,...",2018-12-02 03:00:00+00:00,1.297771
2018-11-19 03:00:00+00:00,"[took, it, easy, after, pain, at, end, of, yes...",2018-11-19 03:00:00+00:00,1.251086
2018-10-04 03:00:00+00:00,"[walked, the, most, so, far, since, the, bad, ...",2018-10-04 03:00:00+00:00,1.224745


In [3]:
health_data.describe()

,Fitness,Fatigue,Rest HR,Active Distance,Active Time,Daily Steps,Log Fitness,Log Fatigue,Active Time / h,Active Distance / km
count,7992.000000,7992.000000,168.000000,49.000000,49.000000,181.000000,7992.000000,7992.000000,49.000000,49.000000
mean,8055.237456,2872.634676,49.291667,11664.907347,2053.959184,2747.232044,3.559841,0.797137,0.570544,11.664907
std,11474.545935,8140.969336,3.265413,18692.170342,3389.953410,2026.764513,0.535319,2.606075,0.941654,18.692170
min,482.219846,0.000005,41.000000,852.870000,192.000000,7.000000,2.683245,-5.312284,0.053333,0.852870
25%,1324.078923,0.088028,47.750000,5268.640000,895.000000,1779.000000,3.121914,-1.055382,0.248611,5.268640
50%,3097.276444,19.631013,50.000000,6113.060000,1024.000000,2369.000000,3.490980,1.292943,0.284444,6.113060
75%,7844.392812,883.819945,51.250000,8943.490000,1380.000000,3012.000000,3.894559,2.946364,0.383333,8.943490
max,48615.888773,55196.774418,57.000000,101526.550000,18139.000000,12757.000000,4.686778,4.741914,5.038611,101.526550


In [5]:
import datetime as dt

from bokeh.io import output_notebook
from bokeh.layouts import row, column
from bokeh.models import Range1d, LinearAxis
from bokeh.plotting import show, figure

from ch2.stoats.names import *

output_notebook()
ymax = health_data[DAILY_STEPS].max()
f = figure(plot_width=1000, plot_height=700, x_axis_type='datetime', 
           x_range=(start, finish), y_range=(0, ymax*1.1))
f.vbar(x=TIME, top=DAILY_STEPS, width=dt.timedelta(hours=20), source=health_data, 
       color='grey', alpha=0.3, legend='Pasos')
f.yaxis.axis_label = 'Pasos / dia'
f.extra_y_ranges = {}

smax = text_data[SCORE].max()
f.extra_y_ranges[SCORE] = Range1d(start=0, end=smax * 1.1)
f.triangle(x=TIME, y=SCORE, source=text_data.loc[text_data[SCORE] > 0], color='red', 
           y_range_name=SCORE, size=10, legend='Dolor (texto)')

tmax = health_data[ACTIVE_TIME_H].max()
f.extra_y_ranges[ACTIVE_TIME_H] = Range1d(start=0, end=tmax * 1.1)
f.circle(x=TIME, y=ACTIVE_TIME_H, source=health_data, color='black', fill_alpha=0, line_width=2, size=8,
         y_range_name=ACTIVE_TIME_H, legend='Tiempo en bici')
f.add_layout(LinearAxis(y_range_name=ACTIVE_TIME_H, axis_label='Tiempo / hr'), 'right')

f.legend.location = (100, 450)

show(f)


Loading BokehJS ...